Now it is time to complicate things and use H2O GBM and RF with grid search, and of course XGBoost.

Thi isi a great article for tunning XGboost: http://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# H2O - GLM, GBM, NN, RF

In [1]:
import pandas as pd
import numpy as np
import time
import csv
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 8
import math

_start_time = time.time()

def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [2]:
import h2o
import time
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator


from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [3]:
#now we load our modified train and test set

tic()
#train = pd.read_csv('./input-data/train_modified.csv', nrows = 500000)
#test = pd.read_csv('./input-data/test_modified.csv', nrows = 500000)
train = pd.read_csv('./input-data/train_modified.csv', nrows=500000,
                    dtype = {'Semana': 'int8',
                            'Agencia_ID': 'uint16',
                            'Canal_ID': 'int8',
                            'Ruta_SAK': 'int32',
                            'Cliente_ID': 'int32',
                            'Producto_ID': 'int32',
                            'log_target':  'float64',
                            'Log_Target_mean_lag1': 'float64',
                            'Log_Target_mean_lag2': 'float64',
                            'Log_Target_mean_lag3': 'float64',
                            'Log_Target_mean_lag4': 'float64',
                            'Lags_sum': 'float64',
                            'pairs_mean':  'float64',
                            'brand': 'uint16',
                            'cluster': 'uint16',
                            'Qty_Ruta_SAK_Bin': 'int8',
                            'ZipCode': 'uint16'}
                   )

                    
test = pd.read_csv('./input-data/test_modified.csv',nrows=500000,
                    dtype = {'id': 'uint32',
                            'Semana': 'int8',
                            'Agencia_ID': 'uint16',
                            'Canal_ID': 'int8',
                            'Ruta_SAK': 'int32',
                            'Cliente_ID': 'int32',
                            'Producto_ID': 'int32',
                            'Log_Target_mean_lag1': 'float64',
                            'Log_Target_mean_lag2': 'float64',
                            'Log_Target_mean_lag3': 'float64',
                            'Log_Target_mean_lag4': 'float64',
                            'Lags_sum': 'float64',
                            'pairs_mean': 'float64',
                            'brand': 'uint16',
                            'cluster': 'uint16',
                            'Qty_Ruta_SAK_Bin': 'int8',
                            'ZipCode': 'uint16'}
                      )
tac()

Time passed: 0.0hour:0.0min:3.0sec


In [4]:
# Connect to a cluster
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,2 hours 1 min
H2O cluster version:,3.10.0.1
H2O cluster version age:,13 days
H2O cluster name:,LOCAL SERVICE
H2O cluster total nodes:,1
H2O cluster free memory:,49.31 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


In [5]:
train_hex = h2o.H2OFrame(train)
test_hex = h2o.H2OFrame(test)

In [6]:
train_hex.describe()

Rows:500,000 Cols:17

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,57,13.9705881,4.5 KB,0.0235195
CXI,Zero Sparse Integers,4,0.9803922,21.7 KB,0.1146471
C1N,1-Byte Integers (w/o NAs),99,24.2647052,2.0 MB,10.6965147
C1S,1-Byte Fractions,8,1.9607844,171.9 KB,0.9078519
C2,2-Byte Integers,24,5.8823530,978.1 KB,5.1661570
C2S,2-Byte Fractions,24,5.8823530,978.5 KB,5.1681787
C4,4-Byte Integers,24,5.8823530,1.9 MB,10.3239790
CUD,Unique Reals,120,29.4117659,4.9 MB,26.3200730
C8D,64-bit Reals,48,11.7647059,7.6 MB,41.2790805



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
10.1.0.4:54321,18.5 MB,500000.0,24.0,408.0
mean,18.5 MB,500000.0,24.0,408.0
min,18.5 MB,500000.0,24.0,408.0
max,18.5 MB,500000.0,24.0,408.0
stddev,0 B,0.0,0.0,0.0
total,18.5 MB,500000.0,24.0,408.0


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17
type,int,int,int,int,int,int,real,real,real,real,real,real,real,int,int,int,int
mins,7.0,1110.0,1.0,1001.0,14432.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0
mean,8.021678,1868.014478,1.316612,1968.989458,1207677.77101,19453.89206,1.71572378921,1.25929790064,0.932080236695,0.924796702283,0.944567677809,4.06074251743,1.72722194655,10.483466,13.66459,0.66068,2030.792262
maxs,9.0,3213.0,11.0,4511.0,9796267.0,49994.0,8.05990833458,8.05990833458,7.83241092719,8.17667277195,8.17667277195,31.329652728,7.74181458113,30.0,30.0,2.0,2050.0
sigma,0.814361953118,961.776062178,1.4054909435,1117.05759979,1552471.45018,18353.2593906,0.775043774119,1.06352973743,1.10114557208,1.10189215805,1.10814333916,3.57595981976,0.698680007449,10.2850601453,7.66254419635,0.866463755225,15.1196160047
zeros,0,0,0,0,0,0,5932,155621,255412,258064,254118,98086,1859,30,0,301304,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7.0,1110.0,7.0,3301.0,15766.0,1216.0,1.09861228867,0.69314718056,1.38629436112,1.09861228867,1.60943791243,4.78749174278,1.27964391678,1.0,2.0,1.0,2008.0
1,8.0,1110.0,7.0,3301.0,15766.0,1216.0,1.79175946923,1.09861228867,0.69314718056,1.38629436112,1.09861228867,4.27666611902,1.27964391678,1.0,2.0,1.0,2008.0
2,7.0,1110.0,7.0,3301.0,15766.0,1238.0,1.09861228867,1.09861228867,0.69314718056,1.38629436112,1.60943791243,4.78749174278,1.19585866875,1.0,2.0,1.0,2008.0


In [7]:
# ----------
# Fit a model on train; using test and validation

# Function for doing class test/train/holdout split
def fit_score(data):
  global gbm0,drf0,glm0,dl0

  val = data[data["C1"] > 8] # Weeks 9
  train = data[data["C1"] <=8] # Weeks 7,8

  print("Training data has",train.ncol,"columns and",train.nrow,"rows, Validation data has",val.nrow,"rows")
  predictors_names = train.names
  predictors_names.remove("C7")

  # Run GBM
  s = time.time()
  
  gbm0 = H2OGradientBoostingEstimator(ntrees=50, # 500 works well
                                      max_depth=15,
                                      learn_rate=0.1)
    

  gbm0.train(x               =predictors_names,
             y               ="C7",
             training_frame  =train,
             validation_frame=val)

  gbm_elapsed = time.time() - s

  # Run DRF
  s = time.time()
    
  drf0 = H2ORandomForestEstimator(ntrees=20, max_depth=30)

  drf0.train(x               =predictors_names,
             y               ="C7",
             training_frame  =train,
             validation_frame=val)
    
  drf_elapsed = time.time() - s 
    
  
  # Run GLM
  s = time.time()

  glm0 = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")
    
  glm0.train(x               =predictors_names,
             y               ="C7",
             training_frame  =train,
             validation_frame=val)

  glm_elapsed = time.time() - s
 
  # Run DL
  s = time.time()

  dl0 = H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=50)
    
  dl0.train(x               =predictors_names,
            y               ="C7",
            training_frame  =train,
            validation_frame=val)
    
  dl_elapsed = time.time() - s
  
  # ----------
  # 4- Score on holdout set & report
  train_r2_gbm = gbm0.model_performance(train).rmse()
  val_r2_gbm  = gbm0.model_performance(val).rmse()
  
  train_r2_drf = drf0.model_performance(train).rmse()
  val_r2_drf  = drf0.model_performance(val).rmse()
 
  train_r2_glm = glm0.model_performance(train).rmse()
  val_r2_glm  = glm0.model_performance(val).rmse()
  
  train_r2_dl = dl0.model_performance(train).rmse()
  val_r2_dl  = dl0.model_performance(val).rmse()
    
  # make a pretty HTML table printout of the results

  header = ["Model", "RMSLE TRAIN", "RMSLE TEST", "Model Training Time (s)"]
  table  = [
            ["GBM", train_r2_gbm, val_r2_gbm, round(gbm_elapsed,3)],
            ["DRF", train_r2_drf, val_r2_drf, round(drf_elapsed,3)],
            ["GLM", train_r2_glm, val_r2_glm, round(glm_elapsed,3)],
            ["DL ", train_r2_dl,  val_r2_dl,  round(dl_elapsed,3) ],
           ]
  h2o.display.H2ODisplay(table,header)
  # --------------

In [8]:
# Split the data (into test & train), fit some models and predict on the holdout data
fit_score(train_hex)

('Training data has', 17, 'columns and', 328667, 'rows, Validation data has', 171333, 'rows')






Model,R2 TRAIN,R2 TEST,Model Training Time (s)
GBM,0.2866214,0.2817645,687.12
DRF,0.1265818,0.2977402,1401.046
GLM,0.4239004,0.4147631,5.4
DL,0.2916710,0.2937356,295.847
